In [2]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
  !pip install --no-deps packaging ninja einops flash-attn xformsers trl peft accelerate bitsandbytes
else:
  !pip install --no-deps xformsers trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-hqujw17b/unsloth_5f958d5590494e9f8d7d50783a6a2815
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-hqujw17b/unsloth_5f958d5590494e9f8d7d50783a6a2815
  Resolved https://github.com/unslothai/unsloth.git to commit d6982c1fe6b814874f2ff989a69e485e6c13ab52
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 11.2 MB/s eta 0:00:00
 

ERROR: Could not find a version that satisfies the requirement xformsers (from versions: none)
ERROR: No matching distribution found for xformsers


In [3]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import AutoTokenizer, TrainingArguments
from datasets import load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [7]:
model, tokenizer = FastLanguageModel.from_pretrained(
  model_name = "unsloth/gemma-7b-bnb-4bit",
  max_seq_length = max_seq_length,
  dtype = dtype,
  load_in_4bit = load_in_4bit,
)

tokenizer = AutoTokenizer.from_pretrained("philschmid/gemma-tokenizer-chatml", use_fast= True)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state = 3407
)

==((====))==  Unsloth 2025.1.5: Fast Gemma patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

In [14]:
dataset = load_dataset("HuggingFaceH4/deita-10k-v0-sft", split=["train_sft","test_sft"])

README.md:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

train_sft-00000-of-00001.parquet:   0%|          | 0.00/140M [00:00<?, ?B/s]

test_sft-00000-of-00001.parquet:   0%|          | 0.00/7.20M [00:00<?, ?B/s]

train_gen-00000-of-00001.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

test_gen-00000-of-00001.parquet:   0%|          | 0.00/6.95M [00:00<?, ?B/s]

Generating train_sft split:   0%|          | 0/9500 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/9500 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [12]:
training_args = TrainingArguments(
    do_eval = True,
    evaluation_strategy = "steps",
    save_strategy = "epoch",
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps = 2,
    warmup_ratio = 0.1,
    num_train_epochs = 1,
    learning_rate = 2e-5,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 =  torch.cuda.is_bf16_supported(),
    lr_scheduler_type = "cosine",
    logging_steps = 100,
    optim = "adamw_8bit",
    seed= 3507,
    output_dir = "./"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset[0],
    eval_dataset = dataset[1],
    max_seq_length = max_seq_length,
    dataset_kwargs = {
        "add_special_tokens": False,
        "append_concat_token": False,
    },
    args = training_args
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [17]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset[0],
    eval_dataset = dataset[1],
    max_seq_length = max_seq_length  ,
    dataset_kwargs = {
        "add_special_tokens": False,
        "append_concat_token": False,
    },
    args = training_args
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

In [19]:
trainer_stats =trainer.train();

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 2
\        /    Total batch size = 8 | Total steps = 1,187
 "-____-"     Number of trainable parameters = 50,003,968
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


AttributeError: 'NoneType' object has no attribute 'attn_bias'

In [18]:
from unsloth import FastLanguageModel
from trl import DPOTrainer
from peft import PeftModel
from transformers import AutoTokenizer, TrainingArguments
from datasets import load_dataset
import torch

In [ ]:
max_seq_length = 1024
dtype = None
load_in_4bit = True

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
  model_name = "/content/chekpoint-593",
  max_seq_length = 2048,
  dtype = dtype,
  load_in_4bit = True,
)